# Aggregate demographic data

This is similar to `4_aggregate_responses.ipynb` but performed separately as we want to aggregate for one school and then v.s. all other schools as one group - and we don't want to breakdown those results any further by any demographic characteristics.

**Note:**
* This method assumes that all pupils in the dataset belong to a school, and that the dataset only contains Northern Devon schools. If either of those conditions changes, you will need to review the code below.
* This method also currently modifies a few categories to be n<10 to confirm censoring is working correctly.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from collections import defaultdict
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd
import sys
sys.path.append('../')
from utilities.response_labels import create_response_label_dict
from create_and_process_data.functions import aggregate_proportions

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    survey = '../data/survey_data'
    synthetic_data = 'synthetic_data_raw.csv'
    aggregate = 'aggregate_demographic.csv'


paths = Paths()

### Import raw data

In [3]:
data = pd.read_csv(os.path.join(paths.survey, paths.synthetic_data))
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen_lab,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
0,4.0,2.0,6.0,NaN,2.0,1.0,2.0,1.0,2.0,5.0,...,Fully,Somewhat helpful,NaN,Not at all,Year 10,Non-FSM,Non-SEN,Ethnic minority,NaN,School E
1,1.0,2.0,1.0,3.0,3.0,2.0,3.0,8.0,NaN,2.0,...,NaN,NaN,NaN,Slightly,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School D
2,2.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,5.0,4.0,...,NaN,NaN,Very comfortable,Not at all,Year 10,Non-FSM,Non-SEN,White British,No,School E
3,2.0,5.0,5.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,...,NaN,NaN,NaN,Mostly,NaN,Non-FSM,Non-SEN,White British,No,School G
4,5.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,...,Slightly,Somewhat helpful,NaN,Not at all,Year 8,Non-FSM,Non-SEN,White British,Yes,School B


### Modify some responses to be n<10

In [4]:
def drop_n_true(boolean_list, n):
    '''
    Drop specified number of 'True' from a list of booleans.

    Parameters
    ----------
    boolean_list : list
        List of True and False values
    n : integer
        Number of True to be replaced with False
    '''
    counter = 0
    to_remove = sum(boolean_list) - n
    # Loop through values of list
    for i in range(len(boolean_list)):
        # If list item is True
        if boolean_list[i]:
            # Increment counter, and set to False if counter is greater than n
            counter += 1
            if counter > to_remove:
                boolean_list[i] = False
    return boolean_list

In [5]:
data.loc[data['school'] ==1, 'transgender'].value_counts(dropna=False).sort_index()

transgender
1.0    24
2.0    24
3.0    20
4.0    28
5.0    28
NaN     4
Name: count, dtype: int64

In [6]:
# Keep 5 responses for category 3
mask = (data['school'] == 1) & (data['transgender'] == 3)
mask = drop_n_true(mask, 5)
data.loc[mask, 'transgender'] = np.nan

# Keep 1 response for category 3
mask = (data['school'] == 1) & (data['transgender'] == 4)
mask = drop_n_true(mask, 1)
data.loc[mask, 'transgender'] = np.nan

# Remove all responses for category 5
mask = (data['school'] == 1) & (data['transgender'] == 5)
data.loc[mask, 'transgender'] = np.nan

data.loc[data['school'] ==1, 'transgender'].value_counts(dropna=False).sort_index()

transgender
1.0    24
2.0    24
3.0     5
4.0     1
NaN    74
Name: count, dtype: int64

### Aggregate data

In [7]:
# Make list of columns that we want to gather responses from
survey_col = ['gender', 'transgender', 'sexual_orientation', 'neurodivergent',
              'birth_parent1', 'birth_parent2', 'birth_you', 'young_carer',
              'care_experience']
council_col = ['year_group', 'fsm', 'sen', 'ethnicity', 'english_additional']
response_col = survey_col + council_col

# Import dictionary which has response options for each variable
labels = create_response_label_dict()

# Add 'NaN': 'No response' to the dictionary for survey columns, and
# 'NaN': 'No data' for the council columns
for col in survey_col:
    labels[col].update({np.nan: 'No response'})
for col in council_col:
    labels[col].update({np.nan: 'No data'})

# Preview two examples
print(labels['birth_parent1'])
print(labels['year_group'])

{1: 'Yes', 2: 'No', 3: "I don't know", nan: 'No response'}
{8: 'Year 8', 10: 'Year 10', nan: 'No data'}


In [8]:
# Initialise list to store results
result_list = list()

# For each of the schools (which we know will all be present at least once
# as we base the school list on the dataset itself)
schools = data['school_lab'].dropna().drop_duplicates().sort_values()
for school in schools:

    # Add label identifying the school as being the current one or now
    data['school_group'] = np.where(data['school_lab'] == school, 1, 0)

    # Loop through each of those groups (current school vs. all other schools)
    for group in [1, 0]:

        # Filter to the group and then aggregate the data
        to_agg = data[data['school_group'] == group]
        res = aggregate_proportions(
            data=to_agg, response_col=response_col, labels=labels)

        # Label with the group
        res['school_lab'] = school
        res['school_group'] = group

        # Append results to list
        result_list.append(res)

# Combine all the results into a single dataframe
result = pd.concat(result_list)

# Hide results where n<10 overall (in addition to item-level already done)
result.loc[result['n_responses'] < 10,
           ['count', 'percentage', 'n_responses']] = np.nan

# Add labels that can use in figures
result['school_group_lab'] = np.where(
    result['school_group'] == 1, 'Your school', 'Other schools')

In [9]:
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,school_group,school_group_lab
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...","[16, 26, 15, 18, 15, 21, 17]","[12.5, 20.3125, 11.71875, 14.0625, 11.71875, 1...",gender,128.0,School A,1,Your school
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...","[24, 24, 5, 1, 0, 74]","[18.75, 18.75, 3.90625, 0.78125, 0.0, 57.8125]",transgender,128.0,School A,1,Your school
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...","[22, 26, 15, 19, 29, 16, 1]","[17.1875, 20.3125, 11.71875, 14.84375, 22.6562...",sexual_orientation,128.0,School A,1,Your school
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]","[41, 33, 34, 20]","[32.03125, 25.78125, 26.5625, 15.625]",neurodivergent,128.0,School A,1,Your school
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]","[38, 43, 42, 5]","[29.6875, 33.59375, 32.8125, 3.90625]",birth_parent1,128.0,School A,1,Your school


### Add group for each measure

In [10]:
# Initialise dictionary of groups
groups = defaultdict(str)

# Define function for adding multiple keys with the same value
def add_keys(value, keys):
    '''
    Add multiple keys with the same value to the dictionary
    Inputs:
    value: String which is the value for all the keys
    keys: Array with the keys
    '''
    groups.update(dict.fromkeys(keys, value))

add_keys('year_group', ['year_group'])
add_keys('fsm', ['fsm'])
add_keys('ethnicity', ['ethnicity'])
add_keys('english_additional', ['english_additional'])
add_keys('gender', ['gender', 'transgender'])
add_keys('care_experience', ['care_experience'])
add_keys('young_carer', ['young_carer'])
add_keys('neuro', ['sen', 'neurodivergent'])
add_keys('birth', ['birth_parent1', 'birth_parent2',
                   'birth_you', 'birth_you_age'])
add_keys('sexual_orientation', ['sexual_orientation'])

# Preview the dictionary
groups

defaultdict(str,
            {'year_group': 'year_group',
             'fsm': 'fsm',
             'ethnicity': 'ethnicity',
             'english_additional': 'english_additional',
             'gender': 'gender',
             'transgender': 'gender',
             'care_experience': 'care_experience',
             'young_carer': 'young_carer',
             'sen': 'neuro',
             'neurodivergent': 'neuro',
             'birth_parent1': 'birth',
             'birth_parent2': 'birth',
             'birth_you': 'birth',
             'birth_you_age': 'birth',
             'sexual_orientation': 'sexual_orientation'})

In [11]:
# Add groups and preview
result['plot_group'] = result['measure'].map(groups)
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,school_group,school_group_lab,plot_group
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...","[16, 26, 15, 18, 15, 21, 17]","[12.5, 20.3125, 11.71875, 14.0625, 11.71875, 1...",gender,128.0,School A,1,Your school,gender
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...","[24, 24, 5, 1, 0, 74]","[18.75, 18.75, 3.90625, 0.78125, 0.0, 57.8125]",transgender,128.0,School A,1,Your school,gender
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...","[22, 26, 15, 19, 29, 16, 1]","[17.1875, 20.3125, 11.71875, 14.84375, 22.6562...",sexual_orientation,128.0,School A,1,Your school,sexual_orientation
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]","[41, 33, 34, 20]","[32.03125, 25.78125, 26.5625, 15.625]",neurodivergent,128.0,School A,1,Your school,neuro
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]","[38, 43, 42, 5]","[29.6875, 33.59375, 32.8125, 3.90625]",birth_parent1,128.0,School A,1,Your school,birth


### Add labels for each measure

In [12]:
# Define labels
labels = {
    'gender': 'Gender',
    'transgender': 'Do you consider yourself to be transgender?',
    'sexual_orientation': 'Sexual orientation',
    'neurodivergent': 'Do you identify as neurodivergent?',
    'young_carer': '''In the last year, have you regularly taken on caring
responsibilities for a family member - e.g. due to illness, disability, mental
health condition or drug/alcohol dependency?''',
    'care_experience': '''Are you or have you ever been in care (living in a
foster placement, residential placement, or private/kinship care)?''',
    'birth_parent1': 'Was birth parent 1 born outside the UK?',
    'birth_parent2': 'Was birth parent 2 born outside the UK?',
    'birth_you': 'Were you born outside the UK?',
    'year_group': 'Year group',
    'fsm': 'Free school meals',
    'sen': 'Special educational needs',
    'ethnicity': 'Ethnicity',
    'english_additional': 'English as an additional language'
}

# Add labels
result['measure_lab'] = result['measure'].map(labels)

# Preview
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,school_group,school_group_lab,plot_group,measure_lab
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...","[16, 26, 15, 18, 15, 21, 17]","[12.5, 20.3125, 11.71875, 14.0625, 11.71875, 1...",gender,128.0,School A,1,Your school,gender,Gender
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...","[24, 24, 5, 1, 0, 74]","[18.75, 18.75, 3.90625, 0.78125, 0.0, 57.8125]",transgender,128.0,School A,1,Your school,gender,Do you consider yourself to be transgender?
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...","[22, 26, 15, 19, 29, 16, 1]","[17.1875, 20.3125, 11.71875, 14.84375, 22.6562...",sexual_orientation,128.0,School A,1,Your school,sexual_orientation,Sexual orientation
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]","[41, 33, 34, 20]","[32.03125, 25.78125, 26.5625, 15.625]",neurodivergent,128.0,School A,1,Your school,neuro,Do you identify as neurodivergent?
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]","[38, 43, 42, 5]","[29.6875, 33.59375, 32.8125, 3.90625]",birth_parent1,128.0,School A,1,Your school,birth,Was birth parent 1 born outside the UK?


In [13]:
# Show the data types and presence of null
types = result.dtypes
null = result.isnull().any()
pd.DataFrame([types, null]).T

,0,1
cat,object,False
cat_lab,object,False
count,object,False
percentage,object,False
measure,object,False
n_responses,float64,False
school_lab,object,False
school_group,int64,False
school_group_lab,object,False
plot_group,object,False


### Save results

In [14]:
result.to_csv(os.path.join(paths.survey, paths.aggregate),
              index=False, na_rep='NULL')